In [ ]:
import intake
import pandas as pd
from pint import UnitRegistry

In [ ]:
ureg = UnitRegistry()
co2_over_c = 44 / 12  # should be able to do in pint. todo: carbonplan[units]?

In [ ]:
def pct_change(new: float, old: float) -> float:
    """calculate percent change between two numbers"""
    return round((new - old) / old, 3)


def load_albedo_data(biome: str) -> pd.DataFrame:
    """Open alebdo data from Drever et al., 2022 paper

    Stored with a few nested layers [described in the metadata]
    Default units are kg C m-2
    """

    cat = intake.open_catalog("../source.yaml")
    raw_data = cat.canada_ecoregion_albedo(
        biome=biome, excel_kwargs={"header": None}
    ).read()

    all_rows = raw_data.values

    statistic = all_rows[0]
    model = all_rows[1]
    scenario = all_rows[2]

    data = []
    for row in all_rows[3:]:
        ecoregion = row[0]
        for idx, value in enumerate(row[1:]):
            true_idx = idx + 1
            data.append(
                (
                    ecoregion,
                    statistic[true_idx],
                    model[true_idx],
                    scenario[true_idx],
                    value,
                )
            )
    df = pd.DataFrame(
        data, columns=["ecoregion", "stat", "model", "scenario", "delta_c"]
    )
    return df

In [ ]:
biomes = ["ENF", "DBF"]

store = {}
for biome in biomes:
    df = load_albedo_data(biome)
    ecoregion = df[(df["ecoregion"] == 117) & (df["scenario"] == "Obs Snow")]
    median_over_kernels = ecoregion.groupby(["stat"]).delta_c.median().to_dict()
    vals = {
        k: round((v * ureg("kg") / ureg.meter**2).to("t/ha") * co2_over_c, 2)
        for k, v in median_over_kernels.items()
    }

    store[biome] = vals

display(store)

### over-crediting

In [ ]:
peak_carbon = 275 * ureg("t/ha")

overcredit_d = {}
for biome, albedo_d in store.items():
    overcredit_d[biome] = {
        k: pct_change(sum([peak_carbon, v]), peak_carbon) for k, v in albedo_d.items()
    }

display(overcredit_d)